# Astra + Vertex GCP Gemini for multimodal ecommerce search

This notebook runs the data loading process

For generating a multimodal embeddi|ng, the image embedding was combined with the name + description embedding.

The products data, with the embedding, will be stored in a Astra collection, in the JSON format.

In [ ]:
!pip install --upgrade --user google-cloud-aiplatform --quiet


In [8]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)

True

## GCP Settings

In [9]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image

In [10]:
GCP_PROJECT_ID = os.environ['GCP_PROJECT_ID']
GCP_REGION = os.environ['GCP_REGION']
vertexai.init(project=GCP_PROJECT_ID, location=GCP_REGION)

In [11]:
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
import numpy as np

# This function generates the embedding for image and text, balancing them at the end.
def get_multimodal_embedding(image_path, text):
    model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")
    image = Image.load_from_file(image_path)
    emb = model.get_embeddings(image=image, contextual_text=text[:768])
    average_embedding = (np.array(emb.image_embedding) + np.array(emb.text_embedding)) / 2   
    return average_embedding.tolist()


## Checking embeddings

In [24]:
#Generating embedding for Text Only
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
import numpy as np

def get_multimodal_embedding_text( text):
    model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")
    emb = model.get_embeddings(contextual_text=text[:768])
    return emb.text_embedding


flip_flop_text_embedding = get_multimodal_embedding_text("Flip Flops")
print(flip_flop_text_embedding[:30])

[0.00495356, 0.0160478521, 0.0111735268, -0.00862137, 0.00317494152, 0.00125397649, 0.0114025204, 0.013161975, -0.00811407622, 0.0263569802, 0.00758765172, -0.0178959183, 0.0181590486, 0.00753808906, 0.00685832137, -0.0288512912, -0.010128486, 0.0130289365, 0.0146572171, -0.00370553276, -0.00142804638, -0.00953206, -0.0176138505, 0.00717362622, 0.00328122173, 8.56568149e-05, -0.0033675083, 0.00208278117, 0.00244230195, -0.00985327084]


In [23]:
#Generating embedding for Image Only
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
import numpy as np

def get_multimodal_embedding_image(image_path):
    model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")
    image = Image.load_from_file(image_path)
    emb = model.get_embeddings(image=image)
    return emb.image_embedding


flip_flop_image_embedding = get_multimodal_embedding_image("../public/img/0f2daee82e04bd8775380196ae032310_black-red-feg-1207-action-shoes-11-original-imaegzyrg3cyecyd.jpg")
print(flip_flop_image_embedding[:50])

[0.0162946675, 0.0255524609, -0.0479237698, -0.00694231782, 0.00302838464, -0.0197078865, 0.00587031757, 0.00964916311, -0.0491338372, 0.00198225025, -0.0152337402, -0.0167719815, -0.013684121, 0.0477043614, 0.0144406445, 0.0306675024, -0.0252390336, 0.0368216038, 0.0111924531, -0.0112337843, 0.0170481224, -0.0345325731, -0.0168440267, 0.023881888, -0.00956977624, -0.00766243553, -0.0300492886, -0.0138860475, -0.00703581516, -0.0361357257, -0.0135501595, 0.00569160795, -0.0354732089, -0.0138120921, -0.00629964517, -0.0119019318, -0.0297375508, -0.0140617676, 0.0049789804, -0.0107155647, -0.0080806138, 0.02510304, -0.000258687593, -0.0183664728, -0.0209119245, -0.0146986721, -0.0517143421, -0.0584810264, -0.014094661, 0.006313344]


In [32]:
# Calculating distance and similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_score =   cosine_similarity(
    np.array(flip_flop_text_embedding).reshape(1,-1), 
    np.array(flip_flop_image_embedding).reshape(1,-1))

print(f"Distância de Coseno:", cosine_similarity_score[0][0])
print(f"Similaridade: {(1+cosine_similarity_score[0][0])/2}")

Distância de Coseno: 0.11246802591642509
Similaridade: 0.5562340129582125


In [34]:
# Tenis embedding
flip_flop_image_embedding = get_multimodal_embedding_image("../public/img/00a3a97fef3c2b377341d240cb9d173a_blue-spo3-sonaxo-10-original-imaegr2jhzaravnp.jpeg")
print(flip_flop_image_embedding[:30])

[-0.0215682145, 0.0525714345, -0.045607809, -0.00770176109, -0.0188693348, -0.00943409465, 0.00218643574, 0.00779017154, -0.00787179731, 0.0251632482, 0.0120792156, -0.00106946, 0.0348235741, 0.0363424681, -0.0236552451, 0.0391172655, 0.0159111191, -0.0389245637, 0.0230476893, -0.00863829441, 0.0518525876, -0.00426573399, -0.0318058953, 0.015824711, -0.0312187038, 0.00197164272, -0.000118429889, 0.00574367028, -0.0250914898, -0.0452114]


In [36]:
flip_flop_text_embedding = get_multimodal_embedding_text("Red Shoes")
print(flip_flop_text_embedding[:30])

[-0.0136976894, 0.0192078408, 0.0100297015, -0.0235605501, 0.00652999, 0.0310531147, 0.0217401423, 0.00060276466, -0.0259038191, -0.00705158478, 0.00748019246, -0.00141556689, 0.0269271079, 0.00575732626, -0.000309936615, -0.0132605517, -0.0080660861, -0.021997638, 0.0192758478, 0.0108571257, -0.0446856506, 0.0163696464, 0.0278088264, 0.00992575288, 0.00683719525, -0.00276321, -0.00530556031, -0.00802841, 0.0242699031, -0.00642605033]


In [35]:
# Tenis embedding
red_shoes_image_embedding = get_multimodal_embedding("../public/img/00a3a97fef3c2b377341d240cb9d173a_blue-spo3-sonaxo-10-original-imaegr2jhzaravnp.jpeg","Red Shoes")
print(red_shoes_image_embedding[:30])

[-0.017632897, 0.0358895827, -0.01778907515, -0.015631129945, -0.006169638815, 0.01080953445, 0.011963289575, 0.004196472466, -0.0168878132, 0.0090558955, 0.00977964763, -0.001242627215, 0.030875316799999997, 0.02105024271, -0.011982599596, 0.01292834385, 0.003922579355000001, -0.030461119499999998, 0.02116171645, 0.0011095297349999998, 0.0035834387000000002, 0.006051984619999999, -0.001998575399999999, 0.012875297595000001, -0.01219073543, -0.00039576494499999997, -0.0027120219660000003, -0.0011424478100000002, -0.00041074305000000054, -0.02581868622]


# Astrapy

In [2]:
from astrapy.db import AstraDB, AstraDBCollection
from astrapy.ops import AstraDBOps

In [3]:
astra_db = AstraDB(token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"), api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"))

In [ ]:
# GCP Gemini Embeddings Dimensions = 1408
# For creating the collection
collection = astra_db.create_collection(collection_name="ecommerce_products", dimension=1408)

In [4]:
# GCP Gemini Embeddings Dimensions = 1408
# Mapping to an existent collection
collection = AstraDBCollection(
    collection_name="ecommerce_products", astra_db=astra_db
)


In [6]:
collection.find_one({})

{'data': {'document': {'_id': '397c966e9c7503ca04c8cfe5a1a5f9c9',
   'product_name': 'Netis PA109',
   'retail_price': 1999,
   'discounted_price': 1599,
   'images': ['http://img5a.flixcart.com/image/router/e/g/t/netis-pa109-1100x1100-imaeczbnbxby4t6k.jpeg',
    'http://img6a.flixcart.com/image/router/e/g/t/netis-pa109-original-imaeczbnbxby4t6k.jpeg',
    'http://img5a.flixcart.com/image/router/e/g/t/netis-pa109-original-imaeczbne7rwuzvg.jpeg'],
   'description': 'Buy Netis PA109 only for Rs. 1599 from Flipkart.com. Only Genuine Products. 30 Day Replacement Guarantee. Free Shipping. Cash On Delivery!',
   'category': 'Computers',
   'specification': [{'key': 'Wireless Speed', 'value': '150 Mbps'},
    {'key': 'Brand', 'value': 'Netis'},
    {'key': 'In The Box', 'value': '1 Antenna PA109'},
    {'key': 'Model', 'value': 'PA109'},
    {'key': 'Type', 'value': 'Range Extenders/Repeaters'},
    {'key': 'Color', 'value': 'Black'},
    {'key': 'Covered in Warranty', 'value': 'Software Driv

# Data Loading

Here, we will read the dataset and prepare it for loading.

The products without price will be discarded.

In [77]:
import pandas as pd
import ast
import json

df = pd.read_csv('./flipkart_com-ecommerce_sample.csv')

def parse_string_to_array(s):
    try:
        return ast.literal_eval(s) if pd.notna(s) else []
    except (SyntaxError, ValueError):
        return []

def extract_category(full_category):
    parse =  parse_string_to_array(full_category)
    if len(parse) > 0:
        return [cat.strip() for cat in parse[0].split(">>")][0]
    else:
        return None

def convert_specification(input_string):
    try:
        json_string = input_string.replace("=>", ":")
        data_dict = json.loads(json_string)
        product_specification = data_dict["product_specification"]
        return product_specification
    except:
        return []

df = df.dropna()
df['image'] = df['image'].apply(parse_string_to_array)
df['category'] = df["product_category_tree"].apply(extract_category)
df['specification'] = df["product_specifications"].apply(convert_specification)
df.head(20)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,category,specification
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,[http://img5a.flixcart.com/image/short/u/4/a/a...,False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",Clothing,[{'key': 'Number of Contents in Sales Package'...
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,[http://img6a.flixcart.com/image/sofa-bed/j/f/...,False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",Furniture,"[{'key': 'Installation & Demo Details', 'value..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,[http://img5a.flixcart.com/image/shoe/7/z/z/re...,False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",Footwear,"[{'key': 'Ideal For', 'value': 'Women'}, {'key..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,[http://img5a.flixcart.com/image/short/6/2/h/a...,False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",Clothing,[{'key': 'Number of Contents in Sales Package'...
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,[http://img5a.flixcart.com/image/pet-shampoo/r...,False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",Pet Supplies,"[{'key': 'Pet Type', 'value': 'Dog'}, {'key': ..."
5,c2a17313954882c1dba461863e98adf2,2016-03-25 22:59:23 +0000,http://www.flipkart.com/eternal-gandhi-super-s...,Eternal Gandhi Super Series Crystal Paper Weig...,"[""Eternal Gandhi Super Series Crystal Paper We...",PWTEB7H2E4KCYUE3,430.0,430.0,[http://img5a.flixcart.com/image/paper-weight/...,False,Key Features of Eternal Gandhi Super Series Cr...,No rating available,No rating available,Eternal Gandhi,"{""product_specification""=>[{""key""=>""Model Name...",Eternal Gandhi Super Series Crystal Paper Weig...,"[{'key': 'Model Name', 'value': 'Gandhi Paper ..."
6,ce5a6818f7707e2cb61fdcdbba61f5ad,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FVVKRBAXHB,1199.0,479.0,[http://img6a.flixcart.com/image/short/p/j/z/a...,False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",Clothing,[{'key': 'Number of Contents in Sales Package'...
7,8542703ca9e6ebdf6d742638dfb1f2ca,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGYGHFUEXN,32157

## Data processing

Here, we will read the dataset to load the data into Astra.

For each row, an embedding will be generated.

The first image of the product will be considered for the image embedding. These images are already downloaded and available at the base_dir.

The skip and load variables was used to limit the loading to and specified range of records.

Initially, I was using insert_many to insert 20 records, but it was complicated to discover which records had problem and couldn't be inserted. Then, I changed it to insert_one. 


In [102]:
%time
from tqdm import tqdm

# Loading all flipcart data to the Vector Table
skip = 5000
load = 6000
batch = 1
docs = []
errors = []
basepath = '../public/img'
for index, row in tqdm(df[skip:load].iterrows(), total=df[skip:load].shape[0], desc=f'Loading with Astrapy'):
    try:
        # Generate the embedding
        emb = get_multimodal_embedding(
                image_path = f"{basepath}/{row['uniq_id']}_{os.path.basename(row['image'][0])}",
                text = f'{row["product_name"]}'
            )
        
        #if an embedding was generated, load the data into Astra
        if emb :
            doc = {
                "_id": row["uniq_id"],
                "product_name": row["product_name"],
                "retail_price": row["retail_price"],
                "discounted_price": row["discounted_price"],
                "images": row["image"],
                "description": row["description"],
                "category": row["category"],
                "specification": row["specification"],
                "$vector": emb}
        
            collection.insert_one(doc)
            
    except Exception as error:
        errors.append(f"Error at IX {index} {error}")


    
print("Finished")
print(f"Errors: {len(errors)}")

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 11.9 µs


Loading with Astrapy: 100%|█████████████| 5000/5000 [5:43:03<00:00,  4.12s/it]

Finished
Errors: 405


In [ ]:
#Inspecting errors
errors

In [100]:
#Inspecting the categories
grouped_df = df.groupby('category').size().reset_index(name='Count')
pd.set_option('display.max_rows', None)
grouped_df

,category,Count
0,883 Police Full Sleeve Solid Men's Jacket,1
1,"ABEEZ Boys, Men, Girls (Black, Pack of 1)",1
2,ANAND ARCHIES Girls Flats,2
3,ANAND ARCHIES Girls Wedges,1
4,ANASAZI Casual 3/4 Sleeve Solid Women's Top,1
5,ATV Pouch for Acer Liquid Z330 (STEEL BLUE),1
6,Abhinl Fashion Cotton Printed Semi-stitched Sa...,1
7,"Adidas IND PRO THI GUA Thigh Guard (White, Blu...",1
8,Ajaero Slim Fit Women's Dark Blue Jeans,2
9,Amita Home Furnishing Cotton Floral Single Bed...,1
